In [24]:
import json
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from datetime import datetime

import itertools
import math


from trueskill import Rating, quality_1vs1, rate_1vs1, setup, global_env, TrueSkill, rate


setup(mu=25.0, sigma=8.333333333333334, beta=4.166666666666667,
      tau=0.08333333333333334, draw_probability=0)


trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%)

In [25]:
with open('C:\REPOS\CSGO\main.json') as f:
  data = json.load(f)





In [ ]:
11287 - len(data)

In [ ]:
#data["2298365"].get("Teams")
#data.keys()
# data["2298365"]

# Prepare Init DF from JSON

In [26]:
Elo = pd.DataFrame(columns=["MatchID","Team1","Team2","Winner"])
for matchid in data.keys():
    t1,t2 = data[matchid].get("Teams")
    s1, s2 = data[matchid].get("Score")
    winner = 1 if s1>s2 else 2
    Elo = Elo.append({"MatchID": matchid, "Team1": t1,
                "Team2": t2, "Winner" : winner }, ignore_index=True)

teams = pd.concat([pd.DataFrame(Elo.Team1.unique()),
                   pd.DataFrame(Elo.Team2.unique())])
teams = teams[0].unique()


In [ ]:
#Elo["Team1"].value_counts()

#FILTER TEAMS BY FREQ

# filtered = filter_by_freq(df=Elo,column="Team1",min_freq=10)
# filtered = filter_by_freq(df=filtered,column="Team2",min_freq=10)

# TRUESKILL

In [ ]:
Elo

In [27]:
# reset or create Dataframe
trueskill_df = Elo.copy()

#label encoding:
le = LabelEncoder()

le.fit(teams)

trueskill_df.Team1 = le.transform(trueskill_df.Team1)
trueskill_df.Team2 = le.transform(trueskill_df.Team2)

trueskill_df['Team1winchance'] = 0.0
trueskill_df['Team2winchance'] = 0.0


trueskill_df['w_ts_after_game'] = Rating()
trueskill_df['l_ts_after_game'] = Rating()
trueskill_df['w_ts_before_game'] = Rating()
trueskill_df['l_ts_before_game'] = Rating()

# trueskill_df['w_ts_before_mu'] = 0
# trueskill_df['w_ts_after_mu'] = 0
# trueskill_df['l_ts_before_mu'] = 0
# trueskill_df['l_ts_after_mu'] = 0
# trueskill_df['w_ts_before_sigma'] = 0.0
# trueskill_df['w_ts_after_sigma'] = 0.0
# trueskill_df['l_ts_before_sigma'] = 0.0
# trueskill_df['l_ts_after_sigma'] = 0.0

n_teams = len(teams)
current_ts = np.full(n_teams, Rating())




In [ ]:
trueskill_df

In [28]:
def win_probability(team1, team2, BETA=4.166666666666667,decimal=3):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return round(ts.cdf(delta_mu / denom),decimal)


In [ ]:
env = TrueSkill()
leaderboard = sorted(current_ts, key=env.expose, reverse=True)
leaderboard

In [29]:
for row in trueskill_df.itertuples():
    idx = row.Index
    w_id = row.Team1 if row.Winner == 1 else row.Team2
    l_id = row.Team2 if row.Winner == 1 else row.Team1

    #get current TS
    w_ts_before = current_ts[w_id]
    l_ts_before = current_ts[l_id]


    #update TS
    w_ts_after, l_ts_after = rate_1vs1(w_ts_before, l_ts_before)


    #  getwinchance
    winchance = win_probability([w_ts_before], [l_ts_before])



    if row.Winner == 1:
        trueskill_df.at[idx, 'Team1winchance'] = winchance
        trueskill_df.at[idx, 'Team2winchance'] = 1-winchance
    if row.Winner == 2:
        trueskill_df.at[idx, 'Team2winchance'] = 1-winchance
        trueskill_df.at[idx, 'Team1winchance'] = winchance


    #save
    trueskill_df.at[idx, 'w_ts_before_game'] = w_ts_before
    trueskill_df.at[idx, 'l_ts_before_game'] = l_ts_before
    trueskill_df.at[idx, 'w_ts_after_game'] = w_ts_after
    trueskill_df.at[idx, 'l_ts_after_game'] = l_ts_after

    # trueskill_df.at[idx, 'w_ts_before_mu'] = w_ts_before.mu
    # trueskill_df.at[idx, 'w_ts_after_mu'] = w_ts_after.mu
    # trueskill_df.at[idx, 'l_ts_before_mu'] = l_ts_before.mu
    # trueskill_df.at[idx, 'l_ts_after_mu'] = l_ts_after.mu
    # trueskill_df.at[idx, 'w_ts_before_sigma'] = w_ts_before.sigma
    # trueskill_df.at[idx, 'w_ts_after_sigma'] = w_ts_after.sigma
    # trueskill_df.at[idx, 'l_ts_before_sigma'] = l_ts_before.sigma
    # trueskill_df.at[idx, 'l_ts_after_sigma'] = l_ts_after.sigma
    current_ts[w_id] = w_ts_after
    current_ts[l_id] = l_ts_after


In [30]:
trueskill_df

,MatchID,Team1,Team2,Winner,Team1winchance,Team2winchance,w_ts_after_game,l_ts_after_game,w_ts_before_game,l_ts_before_game
0,2298365,455,172,2,0.500,0.500,"trueskill.Rating(mu=29.205, sigma=7.195)","trueskill.Rating(mu=20.795, sigma=7.195)","trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.000, sigma=8.333)"
1,2298366,455,137,2,0.632,0.368,"trueskill.Rating(mu=28.318, sigma=7.227)","trueskill.Rating(mu=18.321, sigma=6.496)","trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=20.795, sigma=7.195)"
2,2298361,402,475,1,0.500,0.500,"trueskill.Rating(mu=29.205, sigma=7.195)","trueskill.Rating(mu=20.795, sigma=7.195)","trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.000, sigma=8.333)"
3,2298367,137,172,1,0.470,0.530,"trueskill.Rating(mu=32.072, sigma=6.277)","trueskill.Rating(mu=25.485, sigma=6.258)","trueskill.Rating(mu=28.318, sigma=7.227)","trueskill.Rating(mu=29.205, sigma=7.195)"
4,2298372,111,304,2,0.500,0.500,"trueskill.Rating(mu=29.205, sigma=7.195)","trueskill.Rating(mu=20.795, sigma=7.195)","trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.000, sigma=8.333)"
...,...,...,...,...,...,...,...,...,...,...
10751,2348377,456,484,1,0.466,0.534,"trueskill.Rating(mu=24.437, sigma=1.934)","trueskill.Rating(mu=19.304, sigma=6.342)","trueskill.Rating(mu=24.123, sigma=1.955)","trueskill.Rating(mu=25.000, sigma=8.333)"
10752,2348009,211,56,1,0.397,0.603,"trueskill.Rating(mu=28.355, sigma=0.796)","trueskill.Rating(mu=29.716, sigma=0.798)","trueskill.Rating(mu=28.251, sigma=0.796)","trueskill.Rating(mu=29.820, sigma=0.799)"
10753,2348379,456,306,1,0.381,0.619,"trueskill.Rating(mu=25.032, sigma=1.871)","trueskill.Rating(mu=26.152, sigma=1.080)","trueskill.Rating(mu=24.437, sigma=1.934)","trueskill.Rating(mu=26.341, sigma=1.088)"
10754,2348010,192,347,2,0.553,0.447,"trueskill.Rating(mu=29.361, sigma=0.799)","trueskill.Rating(mu=28.405, sigma=0.811)","trueskill.Rating(mu=29.284, sigma=0.800)","trueskill.Rating(mu=28.484, sigma=0.811)"


In [ ]:
#
#   LABEL ENCODING HELP
#
#
#
#  trueskill_df["Team1"] = le.inverse_transform(
#     trueskill_df["Team1"].astype(int))
# trueskill_df["Team2"] = le.inverse_transform(
#     trueskill_df["Team2"].astype(int))


# trueskill_df.Team1 = le.transform(trueskill_df.Team1)
# trueskill_df.Team2 = le.transform(trueskill_df.Team2)



#   DECODED DF

decoded_df = trueskill_df.copy()

decoded_df["Team1"] = le.inverse_transform(decoded_df["Team1"].astype(int))
decoded_df["Team2"] = le.inverse_transform(decoded_df["Team2"].astype(int))


In [ ]:

decoded_df = decoded_df.sort_index(axis=0, ascending=False)
decoded_df


In [ ]:
teams_df = pd.DataFrame(teams)
teams_df


In [ ]:
def team_exists(team, df=teams):
    res = sum(np.where(df == team,1,0))
    return True if res != 0 else False


In [ ]:
team1 = "NIP"
team2 = "Liquid"
print(team_exists(team1))
print(team_exists(team2))

try:
    print("Team: " + team1 + " trueskill: " +
          str(find_last_trueskill(decoded_df, team1).mu))
except:
    pass

try:
    print("Team: " + team + " trueskill: " +
          str(find_last_trueskill(decoded_df, team2).mu))
except:
    pass


In [ ]:
#
#   SELEKTION HELPER
#
# trueskill_df.iloc[104]

# y = trueskill_df.loc[(halfskill['Team1'] == 455) |
#                      (halfskill['Team2'] == 455)]


In [ ]:
def find_last_trueskill(df,team):

    res = df[(df.Team1 == team) | (df.Team2 == team)].iloc[0]
    mid = res.MatchID
    if team == res.Team1 and res.Winner == 1:
        out = res.w_ts_after_game
    elif team == res.Team2 and res.Winner == 2:
        out = res.w_ts_after_game
    else:
        out = res.l_ts_after_game
    # return out, mid
    return out


In [ ]:
# type(trueskill["w_ts_after_game"].iloc[0])
# trueskill_df.to_csv("TRUESKILL_raw.csv",index=False)


In [ ]:
# trueskill_df["Team1"] = le.inverse_transform(
#     trueskill_df["Team1"])
# trueskill_df["Team2"] = le.inverse_transform(
#     trueskill_df["Team2"])

# g2 = trueskill.Rating(mu=28.279, sigma=0.796)
# navi = trueskill.Rating(mu=29.361, sigma=0.799)


In [ ]:
trueskill_reshape = pd.DataFrame(
    columns=["MatchID","Winner","WinChance", "Winner_TS_before", "Winner_TS_after", "Looser", "Looser_TS_before", "Looser_TS_after"])

for row in trueskill_df.itertuples():
    if row.Winner == 1:
        res = {"MatchID": row.MatchID, "Winner": row.Team1, "WinChance": win_probability([row.w_ts_before_game], [row.l_ts_before_game]), "Winner_TS_before": row.w_ts_before_game,
               "Winner_TS_after": row.w_ts_after_game, "Looser": row.Team2, "Looser_TS_before": row.l_ts_before_game, "Looser_TS_after": row.l_ts_after_game}
    else:
        res = {"MatchID": row.MatchID, "Winner": row.Team2,  "WinChance": win_probability([row.w_ts_before_game], [row.l_ts_before_game]), "Winner_TS_before": row.w_ts_before_game,
               "Winner_TS_after": row.w_ts_after_game, "Looser": row.Team1, "Looser_TS_before": row.l_ts_before_game, "Looser_TS_after": row.l_ts_after_game}
    trueskill_reshape = trueskill_reshape.append(res, ignore_index=True)

#win_probability([row.w_ts_before_game], [row.w_ts_before_game])
trueskill_reshape["Winner"] = le.inverse_transform(
    trueskill_reshape["Winner"].astype(int))
trueskill_reshape["Looser"] = le.inverse_transform(
    trueskill_reshape["Looser"].astype(int))


In [ ]:
trueskill_reshape.to_csv("ExportTrueSkill.csv",index=False)

In [ ]:
trueskill_reshape = pd.DataFrame(
    columns=["MatchID", "Winner", "WinChance", "Winner_mu", "Winner_sigma", "Looser", "Looser_mu", "Looser_sigma"])

for row in trueskill_df.itertuples():
    if row.Winner == 1:
        res = {"MatchID": row.MatchID, "Winner": row.Team1, "WinChance": win_probability([row.w_ts_before_game], [row.l_ts_before_game]), "Winner_mu": row.w_ts_after_game.mu,
               "Winner_sigma": row.w_ts_after_game.sigma, "Looser": row.Team2, "Looser_mu": row.l_ts_after_game.mu, "Looser_sigma": row.l_ts_after_game.sigma}
    else:
        res = {"MatchID": row.MatchID, "Winner": row.Team2, "WinChance": win_probability([row.w_ts_before_game], [row.l_ts_before_game]), "Winner_mu": row.w_ts_after_game.mu,
               "Winner_sigma": row.w_ts_after_game.sigma, "Looser": row.Team1, "Looser_mu": row.l_ts_after_game.mu, "Looser_sigma": row.l_ts_after_game.sigma}
    trueskill_reshape = trueskill_reshape.append(res, ignore_index=True)

#win_probability([row.w_ts_before_game], [row.w_ts_before_game])
trueskill_reshape["Winner"] = le.inverse_transform(
    trueskill_reshape["Winner"].astype(int))
trueskill_reshape["Looser"] = le.inverse_transform(
    trueskill_reshape["Looser"].astype(int))


In [ ]:

trueskill_reshape


In [ ]:
#df['new column name'] = df['column name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')

sumofcorrects = sum(trueskill_reshape['WinChance'].apply(
    lambda x: 1 if x > 0.5 else 0))

sumofnon5050games = sum(trueskill_reshape['WinChance'].apply(
    lambda x: 0 if x == 0.500 else 1))


trueskill_reshape['sigma_thresh_1'] = trueskill_reshape['Winner_sigma'].apply(
    lambda x: 1 if x < 2 else 0)
trueskill_reshape['sigma_thresh_2'] = trueskill_reshape['Looser_sigma'].apply(
    lambda x: 1 if x < 2 else 0)


test_df = trueskill_reshape.apply(lambda row: row[8] == 1 & row[9] == 1,axis=1 )
# sum(trueskill_reshape['sum'])/sum(trueskill_reshape['WinChance'].apply(
#     lambda x: 1 if x == 0.500 else 0))
sigma_df = trueskill_reshape[test_df].copy()
# print(sumofcorrects)
# print(sumofnon5050games)


# print(sumoflowsigmagamesw)
# print(sumoflowsigmagamesl)
# print(sumofcorrects/sumofnon5050games)


In [ ]:
sumofcorrects = sum(sigma_df['WinChance'].apply(
    lambda x: 1 if x > 0.5 else 0))

sumofnon5050games = sum(sigma_df['WinChance'].apply(
    lambda x: 0 if x == 0.500 else 1))

print(sumofcorrects/sumofnon5050games)


In [ ]:
i = 0
BETA = 4.166666666666667
win_probability([trueskill_reshape["Winner_TS_before"].iloc[i]], [trueskill_reshape["Looser_TS_before"].iloc[i]])




In [ ]:
i = 1
print(trueskill_reshape["MatchID"].iloc[i])
print(win_probability([trueskill_reshape["Winner_TS_before"].iloc[i]],[trueskill_reshape["Looser_TS_before"].iloc[i]]))


In [ ]:
trueskill_reshape.set_index("MatchID")



In [ ]:
teams

In [ ]:
#label encoding:

le1 = LabelEncoder()
le2 = LabelEncoder()

filtered.Team1 = le1.fit_transform(filtered.Team1)
filtered.Team2 = le2.fit_transform(filtered.Team2)


In [ ]:
len(filtered["Team1"])


# Elo Rating

In [ ]:
result = filtered.copy()
result.Team1 = le1.inverse_transform(result.Team1)
result.Team2 = le2.inverse_transform(result.Team2)
result.to_csv("ELO_raw.csv",index=False)


In [ ]:
mean_elo = 1500
elo_width = 400
k_factor = 64

In [ ]:
filtered['w_elo_before_game'] = 0
filtered['w_elo_after_game'] = 0
filtered['l_elo_before_game'] = 0
filtered['l_elo_after_game'] = 0
elo_per_season = {}
n_teams = len(teams)
current_elos = np.ones(shape=(n_teams)) * mean_elo

In [ ]:
for row in filtered.itertuples():
    idx = row.Index
    w_id = row.Team1 if row.Winner == 1 else row.Team2
    l_id = row.Team2 if row.Winner == 1 else row.Team1
    
    #get current elos
    w_elo_before = current_elos[w_id]
    l_elo_before = current_elos[l_id]
    #update elos
    w_elo_after, l_elo_after = update_elo(w_elo_before, l_elo_before)

    #save
    filtered.at[idx, 'w_elo_before_game'] = w_elo_before
    filtered.at[idx, 'l_elo_before_game'] = l_elo_before
    filtered.at[idx, 'w_elo_after_game'] = w_elo_after
    filtered.at[idx, 'l_elo_after_game'] = l_elo_after
    current_elos[w_id] = w_elo_after
    current_elos[l_id] = l_elo_after



In [ ]:
def update_elo(winner_elo, loser_elo):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expected_win = expected_result(winner_elo, loser_elo)
    change_in_elo = k_factor * (1-expected_win)
    winner_elo += change_in_elo
    loser_elo -= change_in_elo
    return winner_elo, loser_elo

def expected_result(elo_a, elo_b):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a

def update_end_of_season(elos):
    """Regression towards the mean
    
    Following 538 nfl methods
    https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/
    """
    diff_from_mean = elos - mean_elo
    elos -= diff_from_mean/3
    return elos

    def filter_by_freq(df: pd.DataFrame, column: str, min_freq: int) -> pd.DataFrame:
    """Filters the DataFrame based on the value frequency in the specified column.

    :param df: DataFrame to be filtered.
    :param column: Column name that should be frequency filtered.
    :param min_freq: Minimal value frequency for the row to be accepted.
    :return: Frequency filtered DataFrame.
    """
    # Frequencies of each value in the column.
    freq = df[column].value_counts()
    # Select frequent values. Value is in the index.
    frequent_values = freq[freq >= min_freq].index
    # Return only rows with value frequency above threshold.
    return df[df[column].isin(frequent_values)]

    def find_last_elo(team):

    res = sorted[(sorted.Team1 == team) | (sorted.Team2 == team)].iloc[0]
    mid = res.MatchID
    if team == res.Team1 and res.Winner == 1:
        out = res.w_elo_after_game
    elif team == res.Team2 and res.Winner == 2:
        out = res.w_elo_after_game
    else:
        out = res.l_elo_after_game
    return out, mid


# Elo Rating Sort

In [ ]:
sorted = result.sort_index(ascending=False)
result_team = pd.DataFrame(columns=["MatchID","Team1","Team2","Team1_before","Team2_before"])

for row in result.itertuples():
    if row.Winner == 1:
        team1_before = row.w_elo_before_game
        team2_before = row.l_elo_before_game
    else:
        team2_before = row.w_elo_before_game
        team1_before = row.l_elo_before_game

    result_team = result_team.append(
        {"MatchID" : row.MatchID, "Team1" : row.Team1, "Team2" : row.Team2, "Team1_elo_before" : team1_before, "Team2_elo_before" : team2_before},ignore_index=True)


In [ ]:
result_team.to_csv("ELO_raw.csv",index=False)

In [ ]:
final = pd.concat([pd.DataFrame(sorted.Team1.unique()), pd.DataFrame(sorted.Team2.unique())])
final.columns = ["Team"]
final["Elo"] = 0


In [ ]:
final.head(210)
final.to_csv("ELO.csv")

In [ ]:
team_elo = pd.DataFrame(columns=["Team","Elo","lastID"])

In [ ]:
elo, mid = find_last_elo("G2")
int(mid)

In [ ]:
for row in final.itertuples():
    elo, mid = find_last_elo(row.Team)
    team_elo = team_elo.append({"Team" : row.Team, "Elo" : elo, "lastID" : int(mid) },ignore_index=True)


In [ ]:
team_elo.to_csv("ELO.csv")


In [ ]:
ax  = team_elo["Elo"].plot.kde()